# P2 It's not just a Phase
We will now extend the idea of finding the eigenvalues using the phase kickback trick to the quantum phase estimation (QPE) subroutine by increasing the number of control wires (or estimation wires), and by applying higher powers of the unitary on the target wires.


Author: [Monit Sharma](https://github.com/MonitSharma)
LinkedIn: [Monit Sharma](https://www.linkedin.com/in/monitsharma/)
Twitter: [@MonitSharma1729](https://twitter.com/MonitSharma1729)
Medium : [MonitSharma](https://medium.com/@_monitsharma)

![](https://codebook.xanadu.ai/pics/p2-part-circuit.png)

### Codercise P.2.1. 
Given a unitary matrix $U$, compute the value of a higher power,$U^{2^k}$ . You can use the `matrix_power` function from NumPy's linear algebra library.

In [ ]:
def U_power_2k(unitary, k):
    """ Computes U at a power of 2k (U^2^k)
    
    Args: 
        unitary (array[complex]): A unitary matrix
    
    Returns: 
        array[complex]: the unitary raised to the power of 2^k
    """
    ##################
    # YOUR CODE HERE #
    ##################  
    return np.linalg.matrix_power(unitary,2**k)
            

# Try out a higher power of U
U = qml.T.compute_matrix()
print(U)

U_power_2k(U, 2)
    
   


Since we already have subroutines that implement the Hadamard transform and the QFT, we will implement the middle part of the circuit (the oracle part) that performs the controlled unitary operations on all the estimation wires. This is a helper function that we will use for the end-to-end implementation of the QPE.

### Codercise P.2.2. 
Implement a subroutine that applies the sequence of $U^{2^k}$ unitaries on the target wires controlled on the estimation wires. The function `U_power_2k` from the previous exercise is available for you to use.

In [ ]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def apply_controlled_powers_of_U(unitary):
    """A quantum function that applies the sequence of powers of U^2^k to
    the estimation wires.
    
    Args: 
        unitary (array [complex]): A unitary matrix
    """

    ##################
    # YOUR CODE HERE #
    ##################  
    us = [U_power_2k(unitary, i) for i in reversed(range(len(estimation_wires)))]
    for i in range(len(estimation_wires)):
	    qml.ControlledQubitUnitary(us[i], control_wires=estimation_wires[i],wires=target_wires)


### Codercise P.2.3. 
Implement the QPE subroutine given a unitary, a set of estimation wires, and a set of target wires. The functions defined above (U_power_2k and apply_controlled_powers_of_U ) are available to use. Additionally, the function prepare_eigenvector which prepares an eigenvector of the unitary operator is also given to you below. To implement the QFT, you can make use of PennyLane's template for QFT and qml.adjoint.

In [ ]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """ Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Measurement outcome probabilities on the estimation wires.
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    prepare_eigenvector()
    for i in estimation_wires:
	    qml.Hadamard(wires=i)
    apply_controlled_powers_of_U(unitary)
    qml.adjoint(qml.QFT)(wires=estimation_wires)

    return qml.probs(wires=estimation_wires)
    

U = qml.T.compute_matrix()
print(qpe(U))


### Codercise P.2.4. 
Given the probabilities on the estimation wires, estimate the phase associated with the  gate, when the $T$ eigenvector is prepared in the state $|1\rangle$.

In [ ]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def estimate_phase(probs):
    """Estimate the value of a phase given measurement outcome probabilities
    of the QPE routine.
    
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
    
    Returns:
        float: the estimated phase   
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    increase = 2**-3
    array1 = []

    for _ in range(8):
	    array1.append(_*increase)
    return float(sum(np.array(array1) * probs))

U = qml.T.compute_matrix()

probs = qpe(U)


estimated_phase = estimate_phase(probs)
print(estimated_phase)


Congratulations! You have successfully implemented the QPE subroutine. We will now use PennyLane's implementation of the QPE subroutine to calculate the phase.

### Codercise P.2.5. 
Use PennyLane's template for QPE to calculate the phase of the  gate.

In [ ]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Probabilities on the estimation wires.
    """
    
    prepare_eigenvector()
    
    ##################
    # YOUR CODE HERE #
    ################## 
    #prepare_eigenvector()
    qml.QuantumPhaseEstimation(unitary,target_wires, estimation_wires)
    return qml.probs(wires=estimation_wires)


U = qml.T.compute_matrix()
probs = qpe(U)
print(estimate_phase(probs))
